# process refine bio meta data to parquet



## ⚠️ This Notebook is Not Runnable

**This notebook documents the conversion process used in the original development environment.** The original 3GB TSV file (`metadata_HOMO_SAPIENS.tsv`) is **not** included in this repository due to its size.

**What's available:**
- ✅ The **output** Parquet file (`data/metadata_HOMO_SAPIENS.parquet`, 71MB) is tracked via Git LFS
- ✅ This notebook serves as documentation of how the conversion was performed
- ❌ The source TSV is not here and cannot be re-run

If you need to work with the metadata, use the Parquet file directly (see other notebooks like `detect_potential_clin.ipynb`).

## Context: Why Refine.bio Matters

**The Problem:** NCBI GEO hosts ~1 million human RNA samples with mostly unstructured metadata and no efficient search engine. Each experiment uses different formats and processing pipelines.

**The Solution:** [Refine.bio](https://www.refine.bio/) uniformly processed ~400,000 RNA samples (mostly microarray) with:
- **Normalized expression data**: SCAN + quantile normalization applied uniformly
- **Structured metadata**: Universal `refinebio_*` columns + union of all original GEO columns as a queryable table
- **Quality control**: Pre-filtered for expression coverage

**The Challenge:** Refine.bio's metadata is a 3GB TSV with 6,700+ columns and quirky values. This notebook converts it to an efficient 71MB Parquet file for analysis.

## Why Convert to Parquet?

While I have nothing but respect for the work behind Refine.bio and seriously value what they did, the data formats they used are hard to work with.

The original metadata file is a **3GB TSV** with problematic values (like barcode overflows). Below, I use Polars' lazy evaluation to:
1. **Stream the TSV** without loading into memory (using `scan_csv`)
2. **Infer a robust schema** by scanning all 420K rows during the stream
3. **Fix malformed values** discovered during inference (via `null_values` parameter)
4. **Write to Parquet** in a single pass

**Result:** A **71MB Parquet file** that's ~40× smaller and can be lazily queried without loading into RAM.

**Why this matters for paired data discovery:** Parquet's columnar format makes it efficient to scan 6,700 metadata columns looking for clinical response indicators, without loading the full dataset into memory.

## Why Scan All 420K Rows for Schema Inference?

Polars defaults to inferring types from the **first 100 rows**. This irregular TSV had multiple edge cases:

**Problem 1: Nulls followed by values**
- Some columns had 1,000+ null rows, then `3.14` appeared → should be `Float64`
- With shallow inference: Polars sees only nulls → types as `String` → crashes when it hits the float

**Problem 2: Integers that become floats**
- Column looks like `[1, 2, 3, ...]` for 5,000 rows → Polars infers `Int64`
- Then `3.14` appears at row 10,000 → type mismatch error

**Problem 3: Integer overflow**
- The barcode column had values like `5503934202250110435328` (too large for `Int64`)
- Even with full scan, Polars saw numbers first → inferred `Int64` → **overflow error**
- **Solution:** Added this value to `null_values` parameter to treat overflow as null

**Key insight:** By scanning all 420K rows (`infer_schema_length=420000`), I caught these issues during inference rather than mid-conversion. Understanding Polars' inference behavior allowed me to handle this highly irregular file without errors—and without loading 60GB into memory.

## What is Polars and Why Use It?

**Polars** is a fast dataframe library (alternative to Pandas) designed for:
- **Lazy evaluation**: Build a query plan without executing it (`pl.scan_csv()` instead of `pl.read_csv()`)
- **Memory efficiency**: Stream operations without loading the entire dataset into RAM
- **Speed**: Written in Rust, optimized for columnar data processing

### Key Difference: `read` vs `scan`

```python
# pl.read_csv() - EAGER execution
df = pl.read_csv("huge_file.csv")  
# 💥 Immediately loads entire file into memory (60GB RAM used)
# ✅ Returns DataFrame you can inspect/manipulate immediately
# ✅ Can access .schema, .head(), etc.

# pl.scan_csv() - LAZY execution  
lf = pl.scan_csv("huge_file.csv")
# ✅ Only builds a query plan (~0 MB RAM)
# ✅ Can infer schema during streaming (see infer_schema_length)
# ❌ Cannot access data until you execute (.collect(), .sink_parquet(), etc.)
lf.select(["col1"]).sink_parquet("output.parquet")  # Streams without full load
```

### Schema Inference in Lazy Mode

**Important:** `scan_csv` CAN infer schemas, but it happens during execution (streaming), not upfront:

```python
# This WORKS - schema inference happens while streaming to Parquet
pl.scan_csv(
    "file.tsv",
    infer_schema_length=420000,  # Scan 420K rows to infer types
    null_values=["NA", "overflow_value"]
).sink_parquet("output.parquet")
# ✅ Polars reads in batches, infers types, streams to Parquet
# ✅ Never loads full dataset into memory
```

**Key insight:** You don't need to `read_csv` first to get a schema. The `scan_csv` → `sink_parquet` pipeline infers types on-the-fly during streaming.

## What is a Schema?

A **schema** defines the structure of your data:
- **Column names**: `["sample_id", "age", "tissue", ...]`
- **Data types**: `String`, `Int64`, `Float64`, `Boolean`, etc.

Example:
```python
{"sample_id": String, "age": Int64, "tumor_size": Float64}
```

**Why schemas matter for TSV files:**
- Polars must **infer** types by scanning rows (default: first 100 rows)
- If unusual values appear later, inference fails or mis-types columns
- This TSV had multiple irregularities that required full-length inference to catch

In [ ]:
import json, io, os, sys
import typing as tp
import polars as pl
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.csv as pacsv
from pathlib import Path
import json
from tqdm import tqdm


In [ ]:
import pandas as pd

## helper funcs

In [ ]:
def polars_schema_to_json(df: pl.DataFrame, out: Path) -> None:
    """
    Serialize a Polars DataFrame schema to JSON as {col: dtype_name}.
    dtype_name ∈ {"Utf8","String","Int64","Float64","Boolean",...}
    """
    schema = {name: str(dtype) for name, dtype in df.schema.items()}
    out.write_text(json.dumps(schema, indent=2))
def json_schema_to_polars_dtypes(json_path: Path) -> dict[str, pl.DataType]:
    """
    Read {col: dtype_name} and map to Polars dtypes.
    Unknown names default to Utf8 (conservative).
    """
    name_to_pl = {
        "Utf8": pl.Utf8, "String": pl.Utf8,
        "Int64": pl.Int64, "Float64": pl.Float64, "Boolean": pl.Boolean,
        "Int32": pl.Int32, "Float32": pl.Float32, "Date": pl.Date,
        "Datetime": pl.Datetime, "Time": pl.Time, "Categorical": pl.Categorical,
    }
    schema = json.loads(json_path.read_text())
    return pl.Schema({c: name_to_pl.get(t, pl.Utf8) for c, t in schema.items()})

## paths

In [ ]:
# Original path from development environment (not in this repo)
# root = Path("/mnt/hdd/jesse_archive/stampformer_archive/refine_bio/HOMO_SAPIENS")

# Paths for this repository
from pathlib import Path
root = Path("../data")  # Relative to nbs/ directory

## Stream TSV to Parquet

The approach below uses Polars' lazy evaluation to convert the 3GB TSV to Parquet **without loading it into memory**. 

**How it works:**
1. `scan_csv` creates a lazy query plan (no data loaded yet)
2. `infer_schema_length=420000` tells Polars to scan all rows during execution to infer types
3. `sink_parquet` triggers execution—Polars reads in batches, infers schema, writes to Parquet
4. Memory usage: Only one batch in RAM at a time (vs. 60GB for full load)

In [ ]:
# Updated paths for this repository
data_dir = Path('../data')  # Relative to nbs/ directory

# Source TSV (NOT in this repo - original was 3GB)
tsv_meta_path = data_dir / 'metadata_HOMO_SAPIENS.tsv'  # ⚠️ Does not exist here

# Output files (schema and Parquet ARE in the repo)
schema_path = data_dir / "metadata_HOMO_SAPIENS_schema.json"
parquet_path = data_dir / "metadata_HOMO_SAPIENS.parquet"  # ✅ Available via Git LFS

# Downstream outputs
potential_clin_path = data_dir / 'potential_clin_data.csv'

sep = '\t'

In [ ]:
lazy_data = pl.scan_csv(
    str(tsv_meta_path),
    separator=sep,
    has_header=True,
    infer_schema_length=420000,
    null_values=["", "NA", "NaN", "null", "None",'5503934202250110435328'], 
    ignore_errors=False,
    low_memory=True,
)

In [ ]:
lazy_data.sink_parquet(
    parquet_path,
    compression="zstd",
    compression_level=4,
    row_group_size=64_000,
)

## ✅ Key Takeaways: Efficient Large File Processing

This conversion demonstrates several important Polars concepts:

1. **Lazy evaluation wins**: `scan_csv` → `sink_parquet` never loads the full dataset into memory
   - Memory usage: ~1-2GB (batch size) vs. ~60GB (full eager load)
   
2. **Schema inference during streaming**: Setting `infer_schema_length=420000` tells Polars to scan all rows during the stream to correctly infer types
   - Catches edge cases like nulls-then-floats, ints-that-become-floats, overflow values
   
3. **Problem-solving with null_values**: When I hit the barcode overflow error, I added `'5503934202250110435328'` to `null_values` to treat it as missing data rather than crashing
   - This is faster than pre-cleaning the TSV or manually specifying a schema

4. **Parquet compression**: The result is 40× smaller (71MB vs 3GB) and faster to query
   - `zstd` compression with level 4 balances compression ratio and speed
   - `row_group_size=64_000` optimizes for scanning metadata columns

**Bottom line:** Understanding Polars' lazy evaluation and schema inference allowed me to handle a highly irregular 3GB file without errors or excessive memory usage.